In [1]:
import json
import importlib
import os
current_dir=os.getcwd()
os.chdir("../../")
# print(os.getcwd())
import dist_utilities as dist_ult
# import noise_est_utilities as ne_ult
importlib.reload(dist_ult)
os.chdir(current_dir)
import pickle
from qiskit.result import Result
from qiskit.visualization import plot_histogram, plot_distribution
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
import numpy as np
from copy import deepcopy
from qiskit.result import QuasiDistribution

In [2]:
with open("ideal_counts_hardware.json", "r") as f:
    ideal_counts=json.load(f)
with open("noisy_counts_hardware.json", "r") as f:
    noisy_counts=json.load(f)
with open("noisy_noise_est_dist_hardware.json", "r") as f:
    noisy_noise_est_dist=json.load(f)
with open("ideal_noise_est_counts_hardware.json", "r") as f:
    ideal_noise_est_counts=json.load(f)
print("ideal: ", ideal_noise_est_counts)
print("noise est dist: ", noisy_noise_est_dist)
print("noisy counts: ", noisy_counts)
print("ideal counts: ", ideal_counts)
col_label=list(dict(ideal_noise_est_counts).keys())[0]
# corrected_counts=ne_ult.get_corrected_counts_from_counts(noisy_counts, noisy_noise_est_dist, list(ideal_noise_est_counts.keys())[0])

# print(corrected_counts)

ideal:  {'000000000000001000000000000000': 200000}
noise est dist:  {'000000000000001000000000000000': 0.252125, '000100000000101001000000000000': 3e-05, '000001000000101001000000000000': 2.5e-05, '000000000000001000100000000000': 0.00871, '110100000000001000001000000000': 1e-05, '000000000000001000000010000000': 0.01017, '010010000000101000000000000000': 5.5e-05, '000000000000001010000000000000': 0.019165, '000100000000001000000000000000': 0.0518, '100000000000001000000000000000': 0.00611, '010000000000001000000010100000': 4e-05, '000000000000001000011000000000': 0.0021, '000001000000001000000000000000': 0.030825, '000100000000011100000010000000': 5e-06, '000000000000011000000000000100': 0.000195, '010000000000001000011010000000': 1e-05, '000101000000101000001000000000': 4e-05, '000010000000001000000000000000': 0.01663, '001000000000001000000000000000': 0.00684, '000000000000001000010000000000': 0.00816, '000000000000001010000000000010': 0.00025, '000000000000001000001000000100': 0.00

In [3]:
# plot_histogram([ideal_counts, noisy_counts, corrected_counts], number_to_keep=5, legend=["ideal counts", "noisy counts", "corrected counts"], figsize=(20,20), filename="plot_sim.jpg")


In [4]:
num_qubits=len(list(noisy_counts.items())[0][0])

# if 2**10<2**num_qubits:
num_keep=2**15#num_qubits#2**10
if num_keep>2**num_qubits:
    num_keep=2**num_qubits
noisy_counts_orig=noisy_counts
print("num keep ", num_keep)
noisy_noise_est_dist=dist_ult.convert_to_first_col(noisy_noise_est_dist, col_label)
noisy_noise_est_dist=dist_ult.filter_counts(noisy_noise_est_dist, num_keep)
noisy_counts=dist_ult.filter_counts(noisy_counts, num_keep)
all_keys, noisy_noise_est_dist, noisy_counts=dist_ult.fill_counts_range(noisy_noise_est_dist, noisy_counts, num_keep, num_qubits)

noisy_noise_est_vec=sorted(list(noisy_noise_est_dist.items()), key=lambda x: int(x[0], 2))
noisy_noise_est_vec=[elem[1] for elem in noisy_noise_est_vec]
counts_vec=sorted(list(noisy_counts.items()), key=lambda x: int(x[0], 2))
counts_vec=[elem[1] for elem in counts_vec]
# print(len(noisy_noise_est_vec))
# print(len(counts_vec))
quasi_corrected_counts=dist_ult.get_quasi_corrected(noisy_noise_est_vec, counts_vec)
quasi_corrected_counts=dist_ult.label_quasis(quasi_corrected_counts, all_keys)
quasi_method=QuasiDistribution(quasi_corrected_counts)
quasi_corrected_counts=quasi_method.nearest_probability_distribution().binary_probabilities(num_qubits)
# print(quasi_corrected_counts)
# quasi_corrected_counts={bin(idx)[2::].zfill(num_qubits):val for idx, val in enumerate(quasi_corrected_counts)}



num keep  32768
max exist  1073741823
min exist  0
size before  33555
65536
65536


In [5]:
fig=plot_histogram([ideal_counts, noisy_counts_orig, quasi_corrected_counts], number_to_keep=5, 
               legend=["ideal", "raw", "corrected"], title="GHZ 30 Qubits Quantum Hardware")
fig.savefig("plot_ghz30_counts_hardware_ibm_marrakesh.jpg", bbox_inches="tight")


In [6]:
fig=plot_distribution([ideal_counts, noisy_counts_orig, quasi_corrected_counts], number_to_keep=5, 
                  legend=["ideal", "raw", "corrected"], title="GHZ 30 Qubits Quantum Hardware") #figsize=(15,15))
ax=fig.axes[0]
ax.set_ylabel("Probability")
fig.savefig("plot_ghz30_dist_hardware_ibm_marrakesh.jpg", bbox_inches="tight")

In [7]:
from qiskit.quantum_info import hellinger_fidelity
print(hellinger_fidelity(ideal_counts, quasi_corrected_counts))
print(hellinger_fidelity(ideal_counts, noisy_counts_orig))
print(ideal_counts)
print(quasi_corrected_counts)
print(noisy_counts_orig)

0.9765545368166022
0.2318547864773432
{'000000000000000000000000000000': 100000, '111111111111111111111111111111': 100000}
{'111111111111111111110000000000': 570.462454549119, '111110111111111111111111111111': 784.111583031901, '111011111111111111111111111111': 1423.36230323894, '000011111111111111111111111111': 1890.25708943460, '000000000000000000000000000000': 95645.5461465014, '111111111111111111111111111111': 99686.2604218987}
{'111111111100000000000000100000': 6, '111010000000000000001111111111': 15, '000000000000000000000000000000': 23230, '111111111111111111111111111111': 23141, '111110111111100000000000000000': 13, '111111100000000000000000000000': 1143, '111010100001111111111111111111': 1, '000000011111111111111111111111': 1115, '000000010000000000000000000000': 480, '111111110111111100000000000000': 27, '000000000000001111111111111111': 485, '111111111111111111111111011111': 459, '111111111111110000010000111111': 3, '111111111111111101111111011111': 11, '00001000000000111111